In [ ]:
import random

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular baseline (random)

In [ ]:
from experiments.ranking import Ranking
from experiments.ranking_summarizer import RankingSummarizer
from experiments.ranking import arxiv_waller_ranking

In [ ]:
from experiments.reddit_posts import RedditPosts
from experiments.experiment import Experiment
from experiments.posts_type import Submissions

In [ ]:
def ss_of(year):
    experiment = Experiment(year, WD, 'results', Submissions(), 'original')
    posts = experiment._get_reddit_posts()
    return posts.get_ranked_subreddits()

In [ ]:
def ranked_subreddits_by_year():
    return {year: ss_of(year) for year in range(2012, 2018+1)}

In [ ]:
# subreddits_by_year = ranked_subreddits_by_year()

In [ ]:
subreddits_by_year = {2012: ['democrats',
  'EnoughLibertarianSpam',
  'progressive',
  'racism',
  'GunsAreCool',
  'new_right',
  'conservatives',
  'metacanada',
  'TrueChristian',
  'Conservative'],
 2013: ['democrats',
  'EnoughLibertarianSpam',
  'progressive',
  'racism',
  'GunsAreCool',
  'new_right',
  'conservatives',
  'metacanada',
  'TrueChristian',
  'Conservative'],
 2014: ['democrats',
  'EnoughLibertarianSpam',
  'progressive',
  'racism',
  'GunsAreCool',
  'Christians',
  'new_right',
  'conservatives',
  'metacanada',
  'NoFapChristians',
  'TrueChristian',
  'Conservative'],
 2015: ['democrats',
  'EnoughLibertarianSpam',
  'hillaryclinton',
  'progressive',
  'racism',
  'GunsAreCool',
  'Christians',
  'new_right',
  'conservatives',
  'metacanada',
  'NoFapChristians',
  'TrueChristian',
  'The_Donald',
  'Conservative'],
 2016: ['democrats',
  'EnoughLibertarianSpam',
  'hillaryclinton',
  'progressive',
  'EnoughHillHate',
  'Enough_Sanders_Spam',
  'badwomensanatomy',
  'racism',
  'GunsAreCool',
  'Christians',
  'The_Farage',
  'new_right',
  'conservatives',
  'metacanada',
  'Mr_Trump',
  'NoFapChristians',
  'TrueChristian',
  'The_Donald',
  'Conservative'],
 2017: ['democrats',
  'EnoughLibertarianSpam',
  'hillaryclinton',
  'progressive',
  'BlueMidterm2018',
  'Enough_Sanders_Spam',
  'badwomensanatomy',
  'racism',
  'GunsAreCool',
  'Christians',
  'The_Farage',
  'new_right',
  'conservatives',
  'metacanada',
  'NoFapChristians',
  'TrueChristian',
  'The_Donald',
  'Conservative'],
 2018: ['democrats',
  'EnoughLibertarianSpam',
  'hillaryclinton',
  'progressive',
  'BlueMidterm2018',
  'Enough_Sanders_Spam',
  'badwomensanatomy',
  'racism',
  'GunsAreCool',
  'Christians',
  'The_Farage',
  'new_right',
  'conservatives',
  'metacanada',
  'NoFapChristians',
  'TrueChristian',
  'The_Donald',
  'Conservative']}

In [ ]:
metrics = ['Classic RBO', 'H&H RBO','Two way RBO', 'Kendall Tau', 'AUC ROC',
           'nDCG']

In [ ]:
def generate_random_ranking_results(subreddits):
    ranking_data = {subreddit: random.random() for subreddit in subreddits}
    ranking = Ranking(ranking_data)
    results = ranking.evaluate_ranking_metrics()
    return results

In [ ]:
def summarize_random_rankings(n, subreddits, year=1900):
    results = generate_random_ranking_results(subreddits)
    summarizer = RankingSummarizer(results, 0, year)
    for i in range(1, n):
        results = generate_random_ranking_results(subreddits)
        next_summarizer = RankingSummarizer(results, i, year)
        summarizer = summarizer.union(next_summarizer)

    return summarizer.to_pandas()

In [ ]:
def random_mean(subreddits):
    df = summarize_random_rankings(1000, subreddits)
    random_mean = {metric: [df[metric].mean()] for metric in metrics}
    random_mean = pd.DataFrame(random_mean).round(2)
    return random_mean

In [ ]:
random_mean(arxiv_waller_ranking())

# Generar Gráficos

In [ ]:
import random

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("resultados_raw.csv")

In [ ]:
data.head(100)

In [ ]:
metrics = ['Classic RBO',
 'H&H RBO',
 'Two way RBO',
 'Kendall Tau',
 'AUC ROC',
 'nDCG']

In [ ]:
def df_for_facet_grid(df):
    value_column = pd.Series([], name='value')
    metric_column = pd.Series([], name='metric')
    model_column = pd.Series([], name='model')
    year_column = pd.Series([], name='year')
    
    for metric in metrics: 
        value_column = pd.concat([value_column, df[metric]])
        metric_column = pd.concat([metric_column, pd.Series([metric]*len(df[metric]), name='metric')])
        model_column = pd.concat([model_column, df.model])
        year_column = pd.concat([year_column, df.year])
    
    df = pd.DataFrame({
        'year': year_column,
        'model': model_column,
        'value': value_column,
        'metric': metric_column
    }).sort_values(['year', 'model'])
    return df

df = df_for_facet_grid(data)

In [ ]:
random_mean =  {'Classic RBO': 0.53,
 'H&H RBO': 0.28,
 'Two way RBO': 0.53,
 'Kendall Tau': 0.0,
 'AUC ROC': 0.5,
 'nDCG': 0.77
 }


def plot(metric, df):
    g = sns.catplot(
            data=df, kind="bar",
            x="year", y=metric, hue="model", hue_order=['Cohere', 'truncated_10k', 'pretrained', 'raw'], palette="dark", alpha=.6, height=6, legend=False
        )
    line = plt.axhline(y=random_mean[metric], color='red', linestyle='--', label='baseline (random)')
    g._legend_data['baseline (random)'] = line
    g.add_legend()
    g.legend.set_title("Model")    
    plt.title(f"Models comparison")
    plt.subplots_adjust(top=0.9)

    plt.savefig(f"{metric}.png")

sns.set_theme(style="whitegrid")
for metric in metrics:
    plot(metric, data)


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

metrics = ['Classic RBO', 'H&H RBO', 'Two way RBO', 'Kendall Tau', 'AUC ROC', 'nDCG']


g = sns.FacetGrid(df, col="metric", height=6, col_order=metrics, col_wrap=2)
g.map_dataframe(sns.barplot, x="year", y="value", hue="model",hue_order=['Cohere', 'truncated_10k', 'pretrained', 'raw'], palette="dark", alpha=0.6)
g.set_axis_labels("Year", "Score")
g.set_titles("{col_name}")


def add_axhline(data, **kwargs):
    metric = next(kwargs['metric'])
    plt.axhline(y=random_mean[metric], color='red', linestyle='--', label='baseline (random)')
    
g.map_dataframe(add_axhline, metric=iter(metrics))
g.add_legend()
g.legend.set_title("Model")    

plt.savefig('grouped_plot.png')